In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

In [2]:
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[:30%]"], ## Train test split
    batch_size=-1,
    as_supervised=True,  # Include labels
)

In [3]:
train_ds = tf.image.resize(train_ds,(150,150)) # Resimleri yeniden boyutlandırma
test_ds = tf.image.resize(test_ds,(150,150))

In [4]:
train_labels = to_categorical(train_labels, num_classes = 5) # Etiketleri doğru formata çeviriyoruz
test_labels = to_categorical(test_labels, num_classes = 5)

In [5]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [6]:
# VGG16 modelini yükleme
base_model = VGG16(weights = "imagenet", include_top = False, input_shape = train_ds[0].shape)
base_model.trainable = False

In [7]:
# Preprocessing input ( Giriş verilerini normalleştirmek)
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

In [8]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [9]:
from tensorflow.keras import layers, models

In [10]:
flatten_layers = layers.Flatten()
dense_layer1 = layers.Dense(50, activation = "relu")
dense_layer2 = layers.Dense(20, activation = "relu")
prediction_layer = layers.Dense(5, activation = "softmax")

In [11]:
model = models.Sequential([
    base_model, 
    flatten_layers,
    dense_layer1,
    dense_layer2,
    prediction_layer
])

In [12]:
# Modeli derleyip sığdıralım
from tensorflow.keras.callbacks import EarlyStopping

In [13]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [14]:
es = EarlyStopping(monitor = "val_accuracy", mode = "max", patience = 5, restore_best_weights = True)

In [15]:
model.fit(train_ds, train_labels, epochs = 50, validation_split = 0.2, batch_size = 32, callbacks = [es])

Epoch 1/50
65/65 [==============================] - 75s 1s/step - loss: 1.7649 - accuracy: 0.5017 - val_loss: 1.2455 - val_accuracy: 0.5875
Epoch 2/50
65/65 [==============================] - 76s 1s/step - loss: 0.7309 - accuracy: 0.7567 - val_loss: 1.0866 - val_accuracy: 0.6654
Epoch 3/50
65/65 [==============================] - 71s 1s/step - loss: 0.4975 - accuracy: 0.8243 - val_loss: 1.1273 - val_accuracy: 0.6770
Epoch 4/50
65/65 [==============================] - 73s 1s/step - loss: 0.3410 - accuracy: 0.8745 - val_loss: 1.1671 - val_accuracy: 0.6946
Epoch 5/50
65/65 [==============================] - 73s 1s/step - loss: 0.2151 - accuracy: 0.9231 - val_loss: 1.3502 - val_accuracy: 0.6946
Epoch 6/50
65/65 [==============================] - 71s 1s/step - loss: 0.1529 - accuracy: 0.9479 - val_loss: 1.4449 - val_accuracy: 0.7004
Epoch 7/50
65/65 [==============================] - 77s 1s/step - loss: 0.1200 - accuracy: 0.9620 - val_loss: 1.4307 - val_accuracy: 0.6887
Epoch 8/50
65/65 [==

In [16]:
model.evaluate(test_ds, test_labels)

35/35 [==============================] - 33s 942ms/step - loss: 0.0584 - accuracy: 0.9827


[0.05835213139653206, 0.982742965221405]